In [180]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.tree import export_graphviz
from graphviz import Source
from datetime import datetime
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
%matplotlib inline

In [181]:
# Import the dataset
df = pd.read_csv("loan_final313.csv")
display(df.head())
display(df.shape)

,id,year,issue_d,final_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,loan_condition_cat,interest_rate,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region
0,1077501,2011,01/12/2011,1012015,10.0,RENT,1,Low,24000,1,...,0,10.65,B,2,27.65,5861.071414,5000.00,0.00,162.87,munster
1,1077430,2011,01/12/2011,1042013,0.5,RENT,1,Low,30000,1,...,1,15.27,C,3,1.00,1008.710000,456.46,117.08,59.83,leinster
2,1077175,2011,01/12/2011,1062014,10.0,RENT,1,Low,12252,1,...,0,15.96,C,3,8.72,3003.653644,2400.00,0.00,84.33,cannught
3,1076863,2011,01/12/2011,1012015,10.0,RENT,1,Low,49200,1,...,0,13.49,C,3,20.00,12226.302210,10000.00,0.00,339.31,ulster
4,1075358,2011,01/12/2011,1012016,1.0,RENT,1,Low,80000,1,...,0,12.69,B,2,17.94,3242.170000,2233.10,0.00,67.79,ulster


(887379, 30)

In [182]:
# Choose 10 major features
df0 = df.iloc[:, [8, 4, 21, 10, 12, 20, 6, 16, 18, 23]]
display(df0.head())

,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat,grade_cat
0,24000,10.0,10.65,5000,1,0,1,1,1,2
1,30000,0.5,15.27,2500,2,1,1,2,2,3
2,12252,10.0,15.96,2400,1,0,1,3,2,3
3,49200,10.0,13.49,10000,1,0,1,4,2,3
4,80000,1.0,12.69,3000,2,0,1,4,1,2


# Case 1: y = "interest rate", 9 features in X

In [183]:
# Independent variables X 
X1 = df0.iloc[:, :-1]
# Response variable y
y1 = df0.iloc[:, -1]
display(X1.head())

,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat
0,24000,10.0,10.65,5000,1,0,1,1,1
1,30000,0.5,15.27,2500,2,1,1,2,2
2,12252,10.0,15.96,2400,1,0,1,3,2
3,49200,10.0,13.49,10000,1,0,1,4,2
4,80000,1.0,12.69,3000,2,0,1,4,1


In [184]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X1, y1,random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.9439834118416012


In [185]:
# Show the importance of features
print(clf.feature_importances_)

[2.69803664e-02 1.12795800e-02 9.20897065e-01 2.31973018e-02
 6.17244210e-03 2.04456598e-03 3.71277467e-03 5.52210013e-03
 1.93804105e-04]


* Since interest rate takes up 90\% of importance, try dropping it.

In [120]:
from sklearn.ensemble import RandomForestClassifier
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.9352914427651491
0:00:16.994717


# Case 2 y = "interest rate", 8 features in X (exclude "interest rate")

In [115]:
X2 = X1.drop(['interest_rate'], axis=1)  
display(X2.head())

,annual_inc,emp_length_int,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat
0,24000,10.0,5000,1,0,1,1,1
1,30000,0.5,2500,2,1,1,2,2
2,12252,10.0,2400,1,0,1,3,2
3,49200,10.0,10000,1,0,1,4,2
4,80000,1.0,3000,2,0,1,4,1


In [116]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X2, y1, random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth=15,random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.49814440853327596


In [117]:
# Show the importance of features
print(clf.feature_importances_)

[0.08755582 0.0269363  0.08297295 0.08015142 0.00808996 0.01245245
 0.03112963 0.67071146]


In [118]:
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X2, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.505141381890014
0:00:08.173840


# Case 3 y = "interest rate", 7 features in X (exclude "interest rate" and "interest payment")

In [121]:
X3 = X1.drop(['interest_rate', 'interest_payment_cat'], axis=1)  
display(X3.head())

,annual_inc,emp_length_int,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat
0,24000,10.0,5000,1,0,1,1
1,30000,0.5,2500,2,1,1,2
2,12252,10.0,2400,1,0,1,3
3,49200,10.0,10000,1,0,1,4
4,80000,1.0,3000,2,0,1,4


In [122]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X3, y1, random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth=15,random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.3657250986470562


In [123]:
# Show the importance of features
print(clf.feature_importances_)

[0.23827122 0.07300419 0.22675981 0.28640553 0.03085677 0.03412657
 0.1105759 ]


In [124]:
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X2, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.505141381890014
0:00:08.938667


# Case 4: Clean data, y = "interest rate", 9 features in X

In [125]:
# Data cleansing 
df0 = df.iloc[:, [8, 4, 21, 10, 12, 20, 6, 16, 18, 23]]
df0 = df0[df0.home_ownership_cat < 4]
df0 = df0[df0['purpose_cat'].isin((1,2,3,4,6,7,8))]
display(df0.head())

,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat,grade_cat
0,24000,10.0,10.65,5000,1,0,1,1,1,2
1,30000,0.5,15.27,2500,2,1,1,2,2,3
2,12252,10.0,15.96,2400,1,0,1,3,2,3
3,49200,10.0,13.49,10000,1,0,1,4,2,3
4,80000,1.0,12.69,3000,2,0,1,4,1,2


In [126]:
# Independent variables X 
X4 = df0.iloc[:, :-1]
# Response variable y
y1 = df0.iloc[:, -1]
display(X1.head())

,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat
0,24000,10.0,10.65,5000,1,0,1,1,1
1,30000,0.5,15.27,2500,2,1,1,2,2
2,12252,10.0,15.96,2400,1,0,1,3,2
3,49200,10.0,13.49,10000,1,0,1,4,2
4,80000,1.0,12.69,3000,2,0,1,4,1


In [127]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X4, y1, random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth=10, random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.9484293376053493


In [128]:
# Show the importance of features
print(clf.feature_importances_)

[1.10311610e-04 5.83595217e-05 9.92507385e-01 5.13634936e-04
 5.62264453e-03 8.34337908e-04 7.13071382e-05 7.24674298e-05
 2.09551545e-04]


In [130]:
from sklearn.ensemble import RandomForestClassifier
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X4, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.8857003552274152
0:00:12.418157


# Case 5: Clean data, y = "interest rate", 8 features in X (exclude interest rate)

In [133]:
# Independent variables X 
X5 = df0.iloc[:, :-1]
X5 = X5.drop(['interest_rate'], axis=1)  
display(X5.head())
# Response variable y
y1 = df0.iloc[:, -1]
display(X1.head())

,annual_inc,emp_length_int,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat
0,24000,10.0,5000,1,0,1,1,1
1,30000,0.5,2500,2,1,1,2,2
2,12252,10.0,2400,1,0,1,3,2
3,49200,10.0,10000,1,0,1,4,2
4,80000,1.0,3000,2,0,1,4,1


,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat
0,24000,10.0,10.65,5000,1,0,1,1,1
1,30000,0.5,15.27,2500,2,1,1,2,2
2,12252,10.0,15.96,2400,1,0,1,3,2
3,49200,10.0,13.49,10000,1,0,1,4,2
4,80000,1.0,12.69,3000,2,0,1,4,1


In [134]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X5, y1, random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth=10, random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.5020036683615425


In [135]:
# Show the importance of features
print(clf.feature_importances_)

[0.04356731 0.00155495 0.03728745 0.09126805 0.00695852 0.00710068
 0.02961302 0.78265002]


In [137]:
from sklearn.ensemble import RandomForestClassifier
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X5, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.5035778133779109
0:00:08.743969


# Case 6: Clean data, y = "interest rate", 7 features in X (exclude interest rate)

In [139]:
# Independent variables X 
X6 = df0.iloc[:, :-1]
X6 = X6.drop(['interest_rate','interest_payment_cat'], axis=1)  
display(X6.head())
# Response variable y
y1 = df0.iloc[:, -1]
display(X1.head())

,annual_inc,emp_length_int,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat
0,24000,10.0,5000,1,0,1,1
1,30000,0.5,2500,2,1,1,2
2,12252,10.0,2400,1,0,1,3
3,49200,10.0,10000,1,0,1,4
4,80000,1.0,3000,2,0,1,4


,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat
0,24000,10.0,10.65,5000,1,0,1,1,1
1,30000,0.5,15.27,2500,2,1,1,2,2
2,12252,10.0,15.96,2400,1,0,1,3,2
3,49200,10.0,13.49,10000,1,0,1,4,2
4,80000,1.0,12.69,3000,2,0,1,4,1


In [141]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X6, y1, test_size=0.1, random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(max_depth=10, random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.3711546051867846


In [142]:
# Show the importance of features
print(clf.feature_importances_)

[0.18120205 0.00441156 0.14438446 0.44648636 0.04604712 0.02156484
 0.15590361]


In [143]:
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X6, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.36963618211789834
0:00:09.387543


# Case 7: y as 'loan condition'

In [173]:
# Choose 10 major features
df0 = df.iloc[:, [8, 4, 21, 10, 12, 20, 6, 16, 18, 23]]
display(df0.head())

,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,loan_condition_cat,home_ownership_cat,purpose_cat,interest_payment_cat,grade_cat
0,24000,10.0,10.65,5000,1,0,1,1,1,2
1,30000,0.5,15.27,2500,2,1,1,2,2,3
2,12252,10.0,15.96,2400,1,0,1,3,2,3
3,49200,10.0,13.49,10000,1,0,1,4,2,3
4,80000,1.0,12.69,3000,2,0,1,4,1,2


In [174]:
# Independent variables X 
X7 = df0.drop('loan_condition_cat',axis=1)
# Response variable y
y1 = df0.iloc[:, 5]
display(X1.head())
display(y1.head())

,annual_inc,emp_length_int,interest_rate,loan_amount,term_cat,home_ownership_cat,purpose_cat,interest_payment_cat,grade_cat
0,24000,10.0,10.65,5000,1,1,1,1,2
1,30000,0.5,15.27,2500,2,1,2,2,3
2,12252,10.0,15.96,2400,1,1,3,2,3
3,49200,10.0,13.49,10000,1,1,4,2,3
4,80000,1.0,12.69,3000,2,1,4,1,2


0    0
1    1
2    0
3    0
4    0
Name: loan_condition_cat, dtype: int64

In [175]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X7, y1,random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=0)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.8616781987423652


In [177]:
X77= X7.drop(['interest_rate','interest_payment_cat','grade_cat'], axis=1)
display(X77.head())

,annual_inc,emp_length_int,loan_amount,term_cat,home_ownership_cat,purpose_cat
0,24000,10.0,5000,1,1,1
1,30000,0.5,2500,2,1,2
2,12252,10.0,2400,1,1,3
3,49200,10.0,10000,1,1,4
4,80000,1.0,3000,2,1,4


In [178]:
# Split to train and test set
X_train, X_test, y_train, y_test = train_test_split(X77, y1,random_state=0)
# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=0,max_depth=10)
# Fit the decision tree classifer
clf.fit(X_train, y_train)
# Predict 
y_pred = clf.predict(X_test)
# Accuracy
print(metrics.accuracy_score(y_pred, y_test))

0.9228921093556312


In [179]:
start=datetime.now()
X_train, X_test, y_train, y_test = train_test_split(X77, y1, random_state=0)
forest = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))
print (datetime.now()-start)

0.9233338592260362
0:00:11.158605


In [170]:
# Show the importance of features
print(clf.feature_importances_)

[0.14880072 0.03329932 0.12711101 0.02575495 0.02959051 0.04182033
 0.59362316]
